In [512]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import time
import math
import os
import sys
import matplotlib.pyplot as plt

In [513]:
# download file to tmp/data.txt
!wget -O tmp/data.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-09-25 19:46:55--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8000::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'tmp/data.txt'

     0K .......... .......... .......... .......... ..........  4% 3.16M 0s
    50K .......... .......... .......... .......... ..........  9% 11.1M 0s
   100K .......... .......... .......... .......... .......... 13% 8.51M 0s
   150K .......... .......... .......... .......... .......... 18% 15.6M 0s
   200K .......... .......... .......... .......... .......... 22% 26.1M 0s
   250K .......... .......... .......... .......... .......... 27% 9.42M 0s
   300K .......... .......... .......... .......... .......... 32% 15.2M 0s
  

In [514]:
with open('tmp/data.txt', 'r') as f:
    text = f.read()
print('text length:', len(text))

text length: 1115394


In [515]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [516]:
chars = ["[PAD]", *sorted(list(set(text)))]
vocab_size = len(chars)
print("".join(chars))
print("vocab size:", vocab_size)

[PAD]
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 66


In [517]:
# Create mapping from character to index and vice versa
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[47, 48, 48, 2, 59, 47, 44, 57, 44]
hii there


In [518]:
# store in tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100]) # the 100 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([19, 48, 57, 58, 59,  2, 16, 48, 59, 48, 65, 44, 53, 11,  1, 15, 44, 45,
        54, 57, 44,  2, 62, 44,  2, 55, 57, 54, 42, 44, 44, 43,  2, 40, 53, 64,
         2, 45, 60, 57, 59, 47, 44, 57,  7,  2, 47, 44, 40, 57,  2, 52, 44,  2,
        58, 55, 44, 40, 50,  9,  1,  1, 14, 51, 51, 11,  1, 32, 55, 44, 40, 50,
         7,  2, 58, 55, 44, 40, 50,  9,  1,  1, 19, 48, 57, 58, 59,  2, 16, 48,
        59, 48, 65, 44, 53, 11,  1, 38, 54, 60])


In [519]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [520]:
block_size = 100
train_data[:block_size+1]

tensor([19, 48, 57, 58, 59,  2, 16, 48, 59, 48, 65, 44, 53, 11,  1, 15, 44, 45,
        54, 57, 44,  2, 62, 44,  2, 55, 57, 54, 42, 44, 44, 43,  2, 40, 53, 64,
         2, 45, 60, 57, 59, 47, 44, 57,  7,  2, 47, 44, 40, 57,  2, 52, 44,  2,
        58, 55, 44, 40, 50,  9,  1,  1, 14, 51, 51, 11,  1, 32, 55, 44, 40, 50,
         7,  2, 58, 55, 44, 40, 50,  9,  1,  1, 19, 48, 57, 58, 59,  2, 16, 48,
        59, 48, 65, 44, 53, 11,  1, 38, 54, 60,  2])

In [521]:
a = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = a[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([19]) the target: 48
when input is tensor([19, 48]) the target: 57
when input is tensor([19, 48, 57]) the target: 58
when input is tensor([19, 48, 57, 58]) the target: 59
when input is tensor([19, 48, 57, 58, 59]) the target: 2
when input is tensor([19, 48, 57, 58, 59,  2]) the target: 16
when input is tensor([19, 48, 57, 58, 59,  2, 16]) the target: 48
when input is tensor([19, 48, 57, 58, 59,  2, 16, 48]) the target: 59
when input is tensor([19, 48, 57, 58, 59,  2, 16, 48, 59]) the target: 48
when input is tensor([19, 48, 57, 58, 59,  2, 16, 48, 59, 48]) the target: 65
when input is tensor([19, 48, 57, 58, 59,  2, 16, 48, 59, 48, 65]) the target: 44
when input is tensor([19, 48, 57, 58, 59,  2, 16, 48, 59, 48, 65, 44]) the target: 53
when input is tensor([19, 48, 57, 58, 59,  2, 16, 48, 59, 48, 65, 44, 53]) the target: 11
when input is tensor([19, 48, 57, 58, 59,  2, 16, 48, 59, 48, 65, 44, 53, 11]) the target: 1
when input is tensor([19, 48, 57, 58, 59,  2, 16, 

In [522]:
torch.manual_seed(1337)
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 10 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

# print('----')

# for b in range(batch_size): # batch dimension
#     for t in range(block_size): # time dimension
#         context = xb[b, :t+1]
#         target = yb[b,t]
#         print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([32, 10])
tensor([[57, 42, 47,  2, 54,  6,  2, 59, 47, 44],
        [15, 34, 16, 24, 22, 27, 20, 21, 14, 26],
        [59, 47, 44, 57, 11,  1, 15, 60, 59,  2],
        [48, 61, 44,  2, 52, 44,  2, 59, 47, 64],
        [55, 54, 51, 51, 54,  2, 55, 51, 40, 64],
        [62, 40, 51, 51, 54, 62,  2, 52, 44,  2],
        [48, 57, 58, 59,  2, 26, 60, 57, 43, 44],
        [54,  2, 40,  2, 47, 54, 60, 58, 44,  7],
        [47, 44,  2, 62, 54, 44, 45, 60, 51, 51],
        [14, 41, 54, 60, 59,  2, 59, 47, 44, 48],
        [57, 44,  9,  2, 14, 62, 40, 64,  2, 62],
        [59, 47, 48, 58,  2, 42, 54, 61, 44, 53],
        [22,  2, 52, 44, 40, 53,  2, 59, 54,  2],
        [53, 43,  2, 52, 44, 12,  1, 14, 53, 43],
        [44,  2, 55, 44, 40, 42, 44,  2, 62, 48],
        [43,  2, 45, 57, 54, 52,  2, 47, 48, 58],
        [11,  1, 36, 48, 59, 47,  2, 58, 48, 46],
        [58, 48, 57,  7,  2, 22,  2, 43, 54,  2],
        [44, 40, 51,  2, 52, 64,  2, 59, 57, 60],
        [ 2, 43, 60, 

In [523]:
import numpy as np

# Create a sample NumPy array
arr = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# Define the index 'n' from which you want to start
n = 2

# Get elements from indices n - 5 to n
start_index = max(0, n - 5)  # Ensure the start index is not negative
end_index = n + 1  # Add 1 to include the element at index 'n'

result = arr[start_index:end_index]

print(result)

[0 1 2]


In [524]:
class NGramLanguageModel(nn.Module):
    def __init__(self, vocab_size, n):
        super().__init__()
        self.n = n
        embed_size = vocab_size + 10 * n
        self.token_embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.pos_embedding = nn.Parameter(torch.randn((n, embed_size)))
        self.adding_weight = nn.Parameter(torch.zeros(n))
        self.fc = nn.Linear(embed_size, 200)
        self.final = nn.Linear(200, vocab_size)

    # Create separate function for forward calculation
    def forward_calc(self, x):
        assert len(x.shape) == 2, "input shape should be (batch, time)"

        # if x.shape[1] < self.n:
        #     # pad start with zeros
        #     x = F.pad(x, (self.n - x.shape[1], 0), value=0)

        # print(x.shape)
        # print(x)

        new_x = torch.zeros((x.shape[0], x.shape[1], self.n), dtype=torch.long) - 69
        # -69 is just a random number so if we see it we know something went wrong
        for t in range(x.shape[1]):
            for pos in range(self.n):
                row = x[:, max(0, t - pos) : t + 1]
                row = F.pad(row, (self.n - row.shape[1], 0), value=0)
                row = F.pad(row, (self.n - row.shape[1], 0), value=0)
                # print(row.shape)
                # print(new_x[:, t].shape)
                new_x[:, t] = row 

        # print(f"{new_x.shape=}")
        # print(f"{self.token_embedding(new_x).shape=}")
        # print(f"{self.pos_embedding.shape=}")
        # x = self.token_embedding(x) + self.pos_embedding
        x = torch.add(self.token_embedding(new_x), self.pos_embedding)
        x = F.softmax(self.adding_weight, -1) @ x
        x = self.fc(x)
        x = self.final(x)
        return x

    def forward(self, x, targets=None):
        if targets is None:
            # we're doing inference, so we don't have targets
            # if x is None, we're generating from scratch
            if x is None:
                x = batch_size

            # generate x batches
            if isinstance(x, int):
                x = torch.zeros(x, 1, dtype=torch.long)
            return self.forward_calc(x), None

        else:
            # we're training, so we do have targets
            logits = self.forward_calc(x)
            B, T, C = logits.shape  # batch, time, channel
            logits_flat = logits.view(B * T, C)
            loss = F.cross_entropy(logits_flat, targets.view(B * T))

            return logits_flat, loss

    def generate(self, x, max_len_new, temperature=1.0):
        for _ in range(max_len_new):
            logits, loss = self(x)
            logits_dist = logits[:, -1] / temperature
            probs = F.softmax(logits_dist, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            x = torch.cat([x, next_token], dim=1)

        return x

In [539]:
model = NGramLanguageModel(vocab_size, 20)
# x = (torch.arange(12) + 13).reshape(1, -1).repeat(2, 1)
a = xb[0:2]
print(f"x.shape = {a.shape}")
print(f"x = {a}")
y, loss = model(a)
print(f"y.shape = {y.shape}")
print(f"y = {y}")

x.shape = torch.Size([2, 10])
x = tensor([[11,  1, 16, 54, 52, 44,  7,  2, 52, 64],
        [40, 58, 44,  2, 64, 54, 60,  2, 59, 54]])
y.shape = torch.Size([2, 10, 66])
y = tensor([[[ 0.0637, -0.0793,  0.0446,  ...,  0.0092,  0.0653,  0.1343],
         [ 0.1287, -0.0881,  0.0553,  ..., -0.0048,  0.0702,  0.1399],
         [ 0.1546, -0.0710,  0.0484,  ..., -0.0122,  0.0842,  0.1514],
         ...,
         [ 0.1837, -0.0635,  0.0352,  ..., -0.0606,  0.1079,  0.1717],
         [ 0.2041, -0.0426,  0.0266,  ..., -0.0733,  0.0963,  0.2118],
         [ 0.2249, -0.0212,  0.0295,  ..., -0.0620,  0.0602,  0.2342]],

        [[ 0.0802, -0.0751,  0.0212,  ...,  0.0157,  0.0861,  0.1295],
         [ 0.0418, -0.0541,  0.0109,  ...,  0.0244,  0.0902,  0.1257],
         [ 0.0399, -0.0638,  0.0154,  ...,  0.0012,  0.1290,  0.1350],
         ...,
         [ 0.1071, -0.0797, -0.0535,  ..., -0.0647,  0.1100,  0.1536],
         [ 0.1159, -0.0667, -0.0299,  ..., -0.0510,  0.0842,  0.1360],
         [ 0.097

In [540]:
a = torch.randn((32, 10, 5, 100))
b = torch.randn((5,))
a.shape, b.shape, (b @ a).shape

(torch.Size([32, 10, 5, 100]), torch.Size([5]), torch.Size([32, 10, 100]))

In [541]:
model = NGramLanguageModel(vocab_size, 20)
print(model)
logits, loss = model(xb, yb)
print('logits:', logits.shape)
print('loss:', loss)

print(decode(model.generate(xb, 10)[0].tolist()))

NGramLanguageModel(
  (token_embedding): Embedding(66, 266, padding_idx=0)
  (fc): Linear(in_features=266, out_features=200, bias=True)
  (final): Linear(in_features=200, out_features=66, bias=True)
)
logits: torch.Size([320, 66])
loss: tensor(4.1800, grad_fn=<NllLossBackward>)
:
Come, mycQE-TP.yfh


In [542]:
# train the model
optimizer = optim.AdamW(model.parameters(), lr=0.001)
batch_size = 32
for step in range(5000):
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if step % 500 == 0:
        print(f'step: {step}, loss: {loss.item()}')

print(f'step: {step}, loss: {loss.item()}')

step: 0, loss: 4.1869707107543945


step: 500, loss: 2.6679816246032715
step: 1000, loss: 2.4446654319763184
step: 1500, loss: 2.586050510406494
step: 2000, loss: 2.341681957244873
step: 2500, loss: 2.431805372238159
step: 3000, loss: 2.178189992904663
step: 3500, loss: 2.2504963874816895
step: 4000, loss: 2.470238447189331
step: 4500, loss: 2.5731043815612793
step: 4999, loss: 2.3478097915649414


In [547]:
print(decode(model.generate(torch.tensor([encode("TIO")]), 1000, 0.7)[0].tolist()))

KeyboardInterrupt: 

In [545]:
F.softmax(model.adding_weight)

<ipython-input-545-1dc93fb15ebc>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(model.adding_weight)


tensor([0.0633, 0.0548, 0.0554, 0.0529, 0.0561, 0.0527, 0.0640, 0.0535, 0.0528,
        0.0579, 0.0134, 0.0109, 0.0103, 0.0115, 0.0128, 0.0094, 0.0181, 0.0235,
        0.0687, 0.2579], grad_fn=<SoftmaxBackward>)